<a href="https://colab.research.google.com/github/Akashleena/NuScenes-QA/blob/main/drivelm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir -p "/content/drive/MyDrive/DriveLM_project"
%cd "/content/drive/MyDrive/DriveLM_project"

/content/drive/MyDrive/DriveLM_project


In [ ]:
!git clone https://github.com/bohdanvodianyk/DriveLM.git

fatal: destination path 'DriveLM' already exists and is not an empty directory.


In [ ]:
import time
from IPython.display import HTML, clear_output

while True:
    clear_output(wait=True)
    display(HTML("<script>window.scrollTo(0, document.body.scrollHeight);</script>"))
    time.sleep(60)


KeyboardInterrupt: 

In [ ]:
%cd DriveLM

/content/drive/MyDrive/DriveLM_project/DriveLM


In [ ]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121
!pip install transformers peft
!pip install pycocotools pycocoevalcap
!pip install accelerate tqdm pandas matplotlib


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 477.3/477.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 26.5 MB/s eta 0:00:00


In [ ]:
# Create the correct folder structure first
!mkdir -p /content/drive/MyDrive/DriveLM_project/DriveLM/data/nuscenes

In [ ]:
!gdown --fuzzy "https://drive.google.com/file/d/1DeosPGYeM2gXSChjMODGsQChZyYDmaUz/view"


Downloading...
From (original): https://drive.google.com/uc?id=1DeosPGYeM2gXSChjMODGsQChZyYDmaUz
From (redirected): https://drive.google.com/uc?id=1DeosPGYeM2gXSChjMODGsQChZyYDmaUz&confirm=t&uuid=94546035-9e7a-4a6b-b74f-8ffcbe691dfd
To: /content/drive/MyDrive/DriveLM_project/DriveLM/drivelm_nus_imgs_train.zip
100% 3.48G/3.48G [00:56<00:00, 62.0MB/s]


In [ ]:
!unzip -q /content/drive/MyDrive/DriveLM_project/DriveLM/drivelm_nus_imgs_train.zip \
    -d /content/drive/MyDrive/DriveLM_project/DriveLM/data/nuscenes/

replace /content/drive/MyDrive/DriveLM_project/DriveLM/data/nuscenes/nuscenes/samples/CAM_BACK/n008-2018-08-01-15-52-19-0400__CAM_BACK__1533153528287558.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
y
All


In [ ]:
# Move the samples folder up one level
!mv /content/drive/MyDrive/DriveLM_project/DriveLM/data/nuscenes/nuscenes/samples \
    /content/drive/MyDrive/DriveLM_project/DriveLM/data/nuscenes/

In [ ]:
# Remove the now-empty extra nuscenes folder
!rm -r /content/drive/MyDrive/DriveLM_project/DriveLM/data/nuscenes/nuscenes

In [ ]:
!gdown --fuzzy "https://drive.google.com/file/d/1LK7pYHytv64neN1626u6eTQBy1Uf4IQH/view"

Downloading...
From (original): https://drive.google.com/uc?id=1LK7pYHytv64neN1626u6eTQBy1Uf4IQH
From (redirected): https://drive.google.com/uc?id=1LK7pYHytv64neN1626u6eTQBy1Uf4IQH&confirm=t&uuid=3453508e-4d92-4aa3-9ebe-f9e4af52e291
To: /content/drive/MyDrive/DriveLM_project/DriveLM/v1_0_train_nus.json
100% 193M/193M [00:02<00:00, 90.9MB/s]


In [ ]:
# 1️⃣ Make the QA_dataset_nus folder
!mkdir -p /content/drive/MyDrive/DriveLM_project/DriveLM/data/QA_dataset_nus

# 2️⃣ Move your downloaded v1_0_train_nus.json into it
!mv /content/drive/MyDrive/DriveLM_project/DriveLM/v1_0_train_nus.json \
    /content/drive/MyDrive/DriveLM_project/DriveLM/data/QA_dataset_nus/

# 3️⃣ Unzip your nuScenes subset into correct location
!unzip -q /content/drive/MyDrive/DriveLM_project/DriveLM/drivelm_nus_imgs_train.zip \
    -d /content/drive/MyDrive/DriveLM_project/DriveLM/data/nuscenes/


replace /content/drive/MyDrive/DriveLM_project/DriveLM/data/nuscenes/nuscenes/samples/CAM_BACK/n008-2018-08-01-15-52-19-0400__CAM_BACK__1533153528287558.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: All
A
A
A


In [ ]:
import json
from sklearn.model_selection import train_test_split

# Load QA JSON
with open("/content/drive/MyDrive/DriveLM_project/DriveLM/data/QA_dataset_nus/v1_0_train_nus.json", "r") as f:
    data = json.load(f)

# Split: 70% train, 15% val, 15% test
train_data, temp = train_test_split(list(data.items()), test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp, test_size=0.5, random_state=42)

# Save splits
def save_json(filename, dataset):
    with open(filename, "w") as f:
        json.dump(dict(dataset), f, indent=2)

save_json("/content/drive/MyDrive/DriveLM_project/DriveLM/data/QA_dataset_nus/train.json", train_data)
save_json("/content/drive/MyDrive/DriveLM_project/DriveLM/data/QA_dataset_nus/val.json", val_data)
save_json("/content/drive/MyDrive/DriveLM_project/DriveLM/data/QA_dataset_nus/test.json", test_data)

print("✅ Split complete!")



✅ Split complete!


In [ ]:
import json
import os

# Paths
qa_dir = "/content/drive/MyDrive/DriveLM_project/DriveLM/data/QA_dataset_nus"
multi_frame_dir = "/content/drive/MyDrive/DriveLM_project/DriveLM/data/multi_frame"
os.makedirs(multi_frame_dir, exist_ok=True)

# Function to convert QA format to Vlad's multi_frame format
def convert_to_multiframe(input_path, output_path):
    with open(input_path, "r") as f:
        qa_data = json.load(f)

    multi_frame_data = []
    for scene_id, scene_data in qa_data.items():
        for frame_id, frame_info in scene_data.get("key_frames", {}).items():
            entry = {
                "scene_id": scene_id,
                "frame_id": frame_id,
                "image_paths": frame_info.get("image_paths", {}),
                "QA": frame_info.get("QA", {})
            }
            multi_frame_data.append(entry)

    with open(output_path, "w") as f:
        json.dump(multi_frame_data, f, indent=2)

    print(f"✅ Saved: {output_path} ({len(multi_frame_data)} entries)")

# Convert all three splits
convert_to_multiframe(
    os.path.join(qa_dir, "train.json"),
    os.path.join(multi_frame_dir, "multi_frame_train.json")
)
convert_to_multiframe(
    os.path.join(qa_dir, "val.json"),
    os.path.join(multi_frame_dir, "multi_frame_val.json")
)
convert_to_multiframe(
    os.path.join(qa_dir, "test.json"),
    os.path.join(multi_frame_dir, "multi_frame_test.json")
)


✅ Saved: /content/drive/MyDrive/DriveLM_project/DriveLM/data/multi_frame/multi_frame_train.json (2864 entries)
✅ Saved: /content/drive/MyDrive/DriveLM_project/DriveLM/data/multi_frame/multi_frame_val.json (608 entries)
✅ Saved: /content/drive/MyDrive/DriveLM_project/DriveLM/data/multi_frame/multi_frame_test.json (600 entries)


In [ ]:
# create the directory multi_frame_results
import os

base_results_dir = "/content/drive/MyDrive/DriveLM_project/DriveLM/multi_frame_results"
os.makedirs(base_results_dir, exist_ok=True)
print(f"✅ Created (or already exists): {base_results_dir}")


✅ Created (or already exists): /content/drive/MyDrive/DriveLM_project/DriveLM/multi_frame_results


Preprocessing for QA

In [ ]:
import json
import os

def preprocess_driveLM_json(input_json_path, output_json_path):
    # Load DriveLM's original JSON
    with open(input_json_path, "r") as f:
        data = json.load(f)

    processed = []

    # Loop through scenes
    for scene_token, scene_data in data.items():
        key_frames = scene_data.get("key_frames", {})
        # Loop through key frames
        for frame_token, frame_data in key_frames.items():
            qa = frame_data.get("QA", {})
            img_paths = frame_data.get("image_paths", {})

            # Append tuple exactly as Vlad's code expects
            processed.append((qa, img_paths))

    # Save processed JSON
    with open(output_json_path, "w") as f:
        json.dump(processed, f)

    print(f"✅ Saved {len(processed)} entries to {output_json_path}")

In [ ]:
base_dir = "/content/drive/MyDrive/DriveLM_project/DriveLM/data"

# Train
preprocess_driveLM_json(
    f"{base_dir}/QA_dataset_nus/train.json",
    f"{base_dir}/multi_frame/multi_frame_train.json"
)

# Val
preprocess_driveLM_json(
    f"{base_dir}/QA_dataset_nus/val.json",
    f"{base_dir}/multi_frame/multi_frame_val.json"
)

# Test
preprocess_driveLM_json(
    f"{base_dir}/QA_dataset_nus/test.json",
    f"{base_dir}/multi_frame/multi_frame_test.json"
)

✅ Saved 2864 entries to /content/drive/MyDrive/DriveLM_project/DriveLM/data/multi_frame/multi_frame_train.json
✅ Saved 608 entries to /content/drive/MyDrive/DriveLM_project/DriveLM/data/multi_frame/multi_frame_val.json
✅ Saved 600 entries to /content/drive/MyDrive/DriveLM_project/DriveLM/data/multi_frame/multi_frame_test.json


In [ ]:
import json
with open(f"{base_dir}/multi_frame/multi_frame_train.json", "r") as f:
    processed_data = json.load(f)

print(len(processed_data))
print(type(processed_data[0]))
print(processed_data[0][0].keys())  # QA keys
print(processed_data[0][1].keys())  # camera views

2864
<class 'list'>
dict_keys(['perception', 'prediction', 'planning', 'behavior'])
dict_keys(['CAM_FRONT', 'CAM_FRONT_LEFT', 'CAM_FRONT_RIGHT', 'CAM_BACK', 'CAM_BACK_LEFT', 'CAM_BACK_RIGHT'])


In [ ]:
import json

base_dir = "/content/drive/MyDrive/DriveLM_project/DriveLM/data"

# ---------------- TRAIN ----------------
with open(f"{base_dir}/multi_frame/multi_frame_train.json", "r") as f:
    data = json.load(f)

converted = []
for qa_block, cam_block in data:
    for category in ["perception", "prediction", "planning", "behavior"]:
        if category in qa_block:
            for qa_pair in qa_block[category]:
                converted.append([qa_pair, cam_block])

with open(f"{base_dir}/multi_frame/multi_frame_train.json", "w") as f:
    json.dump(converted, f, indent=2)

print(f"✅ Converted train: {len(converted)} entries")


# ---------------- VAL ----------------
with open(f"{base_dir}/multi_frame/multi_frame_val.json", "r") as f:
    data = json.load(f)

converted = []
for qa_block, cam_block in data:
    for category in ["perception", "prediction", "planning", "behavior"]:
        if category in qa_block:
            for qa_pair in qa_block[category]:
                converted.append([qa_pair, cam_block])

with open(f"{base_dir}/multi_frame/multi_frame_val.json", "w") as f:
    json.dump(converted, f, indent=2)

print(f"✅ Converted val: {len(converted)} entries")


# ---------------- TEST ----------------
with open(f"{base_dir}/multi_frame/multi_frame_test.json", "r") as f:
    data = json.load(f)

converted = []
for qa_block, cam_block in data:
    for category in ["perception", "prediction", "planning", "behavior"]:
        if category in qa_block:
            for qa_pair in qa_block[category]:
                converted.append([qa_pair, cam_block])

with open(f"{base_dir}/multi_frame/multi_frame_test.json", "w") as f:
    json.dump(converted, f, indent=2)

print(f"✅ Converted test: {len(converted)} entries")



✅ Converted train: 265484 entries
✅ Converted val: 56554 entries
✅ Converted test: 55945 entries


In [ ]:
import json

base_dir = "/content/drive/MyDrive/DriveLM_project/DriveLM/data"

def show_one_example(split):
    path = f"{base_dir}/multi_frame/multi_frame_{split}.json"
    with open(path, "r") as f:
        data = json.load(f)

    print(f"📂 {split.upper()} — total entries: {len(data)}")
    example = data[0]  # first item
    qa_block, cam_block = example

    print("\n--- QA block ---")
    for k, v in qa_block.items():
        print(f"{k}: {v}")

    print("\n--- Camera paths ---")
    for k, v in cam_block.items():
        print(f"{k}: {v}")

    print("=" * 60)


# Show one from each split
show_one_example("train")
show_one_example("val")
show_one_example("test")

📂 TRAIN — total entries: 265484

--- QA block ---
Q: What are objects to the back of the ego car?
A: There are many cars and one truck behind the ego car.
C: None
con_up: None
con_down: None
cluster: None
layer: None

--- Camera paths ---
CAM_FRONT: ../nuscenes/samples/CAM_FRONT/n008-2018-07-27-12-07-38-0400__CAM_FRONT__1532707722612404.jpg
CAM_FRONT_LEFT: ../nuscenes/samples/CAM_FRONT_LEFT/n008-2018-07-27-12-07-38-0400__CAM_FRONT_LEFT__1532707722604799.jpg
CAM_FRONT_RIGHT: ../nuscenes/samples/CAM_FRONT_RIGHT/n008-2018-07-27-12-07-38-0400__CAM_FRONT_RIGHT__1532707722620482.jpg
CAM_BACK: ../nuscenes/samples/CAM_BACK/n008-2018-07-27-12-07-38-0400__CAM_BACK__1532707722637558.jpg
CAM_BACK_LEFT: ../nuscenes/samples/CAM_BACK_LEFT/n008-2018-07-27-12-07-38-0400__CAM_BACK_LEFT__1532707722647405.jpg
CAM_BACK_RIGHT: ../nuscenes/samples/CAM_BACK_RIGHT/n008-2018-07-27-12-07-38-0400__CAM_BACK_RIGHT__1532707722628125.jpg
📂 VAL — total entries: 56554

--- QA block ---
Q: What are objects to the back l

In [ ]:
import json
import os

base_dir = "/content/drive/MyDrive/DriveLM_project/DriveLM/data"
abs_base = os.path.join(base_dir, "nuscenes")  # where samples/ is located

def fix_paths(json_path):
    with open(json_path, "r") as f:
        data = json.load(f)

    fixed = []
    for qa_block, cam_block in data:
        fixed_cam_block = {}
        for cam, rel_path in cam_block.items():
            # Remove "../nuscenes/" if present
            clean_rel = rel_path.replace("../nuscenes/", "")
            # Make absolute path
            abs_path = os.path.join(abs_base, clean_rel)
            fixed_cam_block[cam] = abs_path
        fixed.append([qa_block, fixed_cam_block])

    with open(json_path, "w") as f:
        json.dump(fixed, f, indent=2)

    print(f"✅ Fixed paths in {json_path}")

# Run for all splits
for split in ["multi_frame_train.json", "multi_frame_val.json", "multi_frame_test.json"]:
    fix_paths(os.path.join(base_dir, "multi_frame", split))


✅ Fixed paths in /content/drive/MyDrive/DriveLM_project/DriveLM/data/multi_frame/multi_frame_train.json
✅ Fixed paths in /content/drive/MyDrive/DriveLM_project/DriveLM/data/multi_frame/multi_frame_val.json
✅ Fixed paths in /content/drive/MyDrive/DriveLM_project/DriveLM/data/multi_frame/multi_frame_test.json


Training

In [ ]:
from torch.utils.data import Dataset
from torchvision.io import read_image
from transformers import T5Tokenizer
from torchvision import transforms
import torch
import json
import os
import time
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer, GPT2LMHeadModel, GPT2TokenizerFast
from torchvision.models import vit_b_32
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
import argparse
from peft import LoraConfig, get_peft_model
from collections import namedtuple
import matplotlib.pyplot as plt
import pandas as pd
from copy import deepcopy
from tqdm import tqdm


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
TEXT_MAX_LENGTH = 463
VIT_HIDDEN_STATE = 768
VIT_SEQ_LENGTH = 49
GPT_N_EMBED = 1024

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    print(
        f"Trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )



class DriveVLM(nn.Module):

    def __init__(self, config):

        super().__init__()

        # Make tokenizer and text model
        if config.lm == 'T5':
            self.model = T5ForConditionalGeneration.from_pretrained('google-t5/t5-base')
            hidden_size = self.model.config.d_model
        else:
            self.model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
            hidden_size = GPT_N_EMBED


        # Only if we are perfoming LoRA finetuning on T5
        if config.lora:

            if config.lm == 'T5':
              tm = ['q', 'v']
            else:
              tm = ['c_attn']

            # Create LoRA model
            lora_config = LoraConfig(
                r=config.lora_dim,
                lora_alpha=config.lora_alpha,
                lora_dropout=config.lora_dropout,
                bias='none',
                target_modules=tm
            )
            self.model = get_peft_model(self.model, lora_config)

        # Freeze model weights if needed
        if config.freeze_lm:

            for param in self.model.parameters():
                param.requires_grad = False

        print('Trainable Parameters for LM model:')
        print_trainable_parameters(self.model)

        # Create instance for multi-view processor
        self.mvp = self.MultiViewProcessor(config.gpa_hidden_size, hidden_size, config.lm, freeze=True)

    class MultiViewProcessor(nn.Module):

        def __init__(self, gpa_hidden_size, hidden_size, lm, freeze=False):

            super().__init__()

            # Use ViT for image embeddings
            self.img_model = vit_b_32(weights='DEFAULT')
            self.lm = lm

            # Modal embedding to distinguish between image and text
            self.modal_embeddings = nn.Embedding(2, hidden_size)
            self.modal_embeddings.weight.data.normal_(mean=0.0, std=0.02)

            # If we are freezing the CLIP embeddings
            if freeze:
                for param in self.img_model.parameters():
                    param.requires_grad = False

            # Set matrices based on MIVC paper
            self.w = nn.Linear(in_features=gpa_hidden_size, out_features=1)
            self.Z = nn.Sequential(
                nn.Linear(in_features=VIT_HIDDEN_STATE * VIT_SEQ_LENGTH, out_features=gpa_hidden_size, bias=False),
                nn.Tanh()
            )
            self.G = nn.Sequential(
                nn.Linear(in_features=VIT_HIDDEN_STATE * VIT_SEQ_LENGTH, out_features=gpa_hidden_size, bias=False),
                nn.Sigmoid()
            )

            if self.lm != 'T5':
              self.img_projection_layer = nn.Linear(in_features=VIT_HIDDEN_STATE, out_features=hidden_size)


        def gpa(self, img_embeddings):

            """"
            Calculates the gated-pooling attention score for the image embeddings
            :param img_embeddings: (6x768) dimensional
            :return single embedding of size (768,)
            """

            # Get weights for gated pooling attention
            gpa_weights = torch.softmax(self.w(self.Z(img_embeddings) * self.G(img_embeddings)), dim=0  )

            # Take a linear combination of all the image embeddings
            fused_embeddings = torch.sum(gpa_weights * img_embeddings, dim=0)

            return fused_embeddings

        def get_img_embedding(self, imgs):

            N = imgs.shape[0]

            # Process into patches (N x 6 x 49 x H)
            merged_embedding = torch.stack([self.img_model._process_input(img) for img in imgs], dim=0)

            # Concatenate the batch class tokens -> (N, 6, 50, H)
            batch_class_tokens = self.img_model.class_token.expand(merged_embedding.shape[1], -1, -1).repeat(N, 1, 1, 1)
            merged_embedding = torch.cat([batch_class_tokens, merged_embedding], dim=2)

            # Add positional embeddings and remove class token -> (N, 6, 49, H)
            merged_embedding += self.img_model.encoder.pos_embedding.repeat(N, 1, 1, 1)
            merged_embedding = merged_embedding[:, :, 1:]

            # Get merged embedding and reshape to 2D embedding -> (N, 1, 49, H)
            merged_embedding = torch.stack([self.gpa(embedding.flatten(start_dim=1)).reshape(VIT_SEQ_LENGTH,
                                            VIT_HIDDEN_STATE) for embedding in merged_embedding], dim=0)

            # Project to VL dimension -> (1, 49, H) (H is 512 for t5-small, 768 for t5-base)
            if self.lm != 'T5':
              merged_embedding = self.img_projection_layer(merged_embedding)

            # Add modal type embedding to merged embedding
            merged_embedding += self.modal_embeddings(
                torch.ones((1, merged_embedding.shape[1]), dtype=torch.int, device=device))

            return merged_embedding

        def forward(self, text_enc, imgs, text_model):

            # Get the image embeddings (N x 1 x 49 x H)
            imgs_embedding = self.get_img_embedding(imgs)

            # Get the text embeddings (N x S x H)
            text_embeddings = text_model.get_input_embeddings()(text_enc)

            # Add modal embeddings to text
            text_embeddings += self.modal_embeddings(torch.zeros((1, text_embeddings.shape[1]), dtype=torch.int,
                                                                 device=device))

            # Concatenate embeddings -> (1 x S x 512)
            merged_embedding = torch.cat([text_embeddings, imgs_embedding], dim=1)

            return merged_embedding

    def forward(self, text_enc, imgs, labels=None):

        # Get the merged embeddings
        merged_embedding = self.mvp(text_enc, imgs, self.model)

        # If training include the labels
        return self.model(inputs_embeds=merged_embedding, labels=labels)

In [ ]:
class MultiFrameDataset(Dataset):

    def __init__(self, input_file, tokenizer, transform=None):
        with open(input_file) as f:
            self.data = json.load(f)

        self.tokenizer = tokenizer
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Get the question and answer at the idx
        qa, img_path = self.data[idx]
        img_path = [os.path.join('DriveLM', p)
                    for p in list(img_path.values())]

        q_text, a_text = qa['Q'], qa['A']
        q_text = f"Question: {q_text} Answer:"

        # Concatenate images into a single tensor
        imgs = [self.transform(read_image(p).float()).to(device) for p in img_path]
        imgs = torch.stack(imgs, dim=0)

        return q_text, imgs, a_text

    def collate_fn(self, batch):

        q_texts, imgs, a_texts = zip(*batch)
        imgs = torch.stack(list(imgs), dim=0)

        encodings = self.tokenizer(q_texts, padding=True, return_tensors="pt").input_ids.to(device)
        labels = self.tokenizer(a_texts, padding=True, return_tensors='pt').input_ids.to(device)

        return encodings, imgs, labels

In [ ]:
Config = namedtuple('Instance', ['batch_size', 'learning_rate',
                                 'weight_decay', 'num_workers',
                                 'epochs', 'custom_train', 'gpa_hidden_size',
                                 'lora', 'lora_dim', 'lora_alpha', 'lora_dropout',
                                 'load_checkpoint', 'file_checkpoint', 'checkpoint_frequency',
                                 'freeze_lm', 'lm'])

# We recommend keep all hyperparameters the same besides the following
# file_checkpoint -> Checkpoint folder stored in multi_frame_results folder
# load_checkpoint -> Use if you want to resume training from a checkpoint stored in multi_frame_results
config = Config(
    batch_size = 4,
    learning_rate = 1e-4,
    weight_decay = 0.05,
    num_workers = 0,
    epochs = 6,
    custom_train = True,
    gpa_hidden_size = 128,
    lora = False,
    lora_dim = 64,
    lora_alpha = 32,
    lora_dropout = 0.05,
    load_checkpoint = False,
    file_checkpoint = '20240301-053312',
    checkpoint_frequency = 15000,
    freeze_lm = True,
    lm = 'T5'
)

In [ ]:
# Set working directory
%cd '/content/drive/MyDrive/DriveLM_project/DriveLM'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def save_model(state_dict, model_name):
    """Save only model weights (state_dict) to save space."""
    save_path = os.path.join(
        '/content/drive/MyDrive/DriveLM_project/DriveLM',
        'multi_frame_results', timestr, model_name + '.pth'
    )
    torch.save(state_dict, save_path)
    print(f"💾 Saved checkpoint: {save_path}")

def load_model(model, checkpoint_name):
    """Load model weights into an existing model."""
    load_path = os.path.join(
        '/content/drive/MyDrive/DriveLM_project/DriveLM',
        'multi_frame_results', timestr, checkpoint_name + '.pth'
    )
    state_dict = torch.load(load_path, map_location=device)
    model.load_state_dict(state_dict)
    print(f"✅ Loaded checkpoint: {load_path}")
    return model

def val_model(dloader, val_model):
    val_model.eval()
    val_loss = 0
    for idx, (inputs, imgs, labels) in tqdm(enumerate(dloader), total=len(dloader)):
        outputs = val_model(inputs, imgs, labels)
        val_loss += outputs.loss.item()
    return val_loss / len(val_dataloader)

def save_stats(train_loss, val_loss, epochs, lr):
    stats_dict = {
        'losses': losses,
        'val losses': val_losses,
        'min train loss': train_loss,
        'min val loss': val_loss,
        'epochs': epochs,
        'learning rate': lr,
        'LM': 'T5-Base',
        'Image Embedding': 'Patch'
    }
    with open(os.path.join('/content/drive/MyDrive/DriveLM_project/DriveLM',
                           'multi_frame_results', timestr, 'stats.json'), 'w') as f:
        json.dump(stats_dict, f)

def plot_loss(training_loss, val_loss):
    num_epochs = len(training_loss)
    plt.plot(range(1, num_epochs + 1), training_loss, label='Training Loss')
    plt.plot(range(1, num_epochs + 1), val_loss, label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Num epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(os.path.join('/content/drive/MyDrive/DriveLM_project/DriveLM',
                             'multi_frame_results', timestr, 'loss.png'))


#

/content/drive/MyDrive/DriveLM_project/DriveLM


In [ ]:
 ========================
# TRAINING LOOP
# ========================

from torch.cuda.amp import autocast, GradScaler

def custom_train(train_loss, val_loss, best_model, epochs, learning_rate):
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9, last_epoch=-1)
    scaler = GradScaler()

    checkpoint_every_steps = 5000  # mid-epoch save frequency

    for epoch in range(epochs, config.epochs):
        print(f'-------------------- EPOCH {epoch} ---------------------')
        model.train()
        epoch_loss = 0

        for step, (inputs, imgs, labels) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):

            with autocast():
                outputs = model(inputs, imgs, labels)
                loss = outputs.loss

            epoch_loss += loss.item()

            if step % config.checkpoint_frequency == 0:
                print(f'\nLoss: {loss.item()}')

            # backprop with AMP
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

            # mid-epoch checkpoint save
            if step > 0 and step % checkpoint_every_steps == 0:
                ckpt_name = f"epoch{epoch}_step{step}"
                save_model(model.state_dict(), ckpt_name)

        # end of epoch logging
        epoch_train_loss = epoch_loss / len(train_dataloader)
        losses.append(epoch_train_loss)
        epoch_val_loss = val_model(val_dataloader, model)
        val_losses.append(epoch_val_loss)

        # best model tracking
        if not val_loss or epoch_val_loss < val_loss:
            val_loss = epoch_val_loss
            best_model = deepcopy(model.state_dict())
        if not train_loss or epoch_train_loss < train_loss:
            train_loss = epoch_train_loss

        scheduler.step()
        print(f'Training Loss: {epoch_train_loss} | Validation Loss: {epoch_val_loss}')

        # save end-of-epoch checkpoint
        save_model(best_model, 'latest_model')
        epochs += 1
        save_stats(train_loss, val_loss, epochs, scheduler.get_last_lr()[0])

    plot_loss(losses, val_losses)
    return train_loss, val_loss

# ========================
# EXPERIMENT LOGGING
# ========================
def save_experiment(statistics):
    trial_dict = {
        'Model name': [timestr],
        'Learning rate': [config.learning_rate],
        'Weight decay': [config.weight_decay],
        'Batch size': [config.batch_size],
        'Epochs': [config.epochs],
        'LoRA finetuning': [config.lora],
        'GPA Hidden Size': [config.gpa_hidden_size],
        'LoRA Dimension': [config.lora_dim],
        'LoRA Alpha': [config.lora_alpha],
        'LoRA Dropout': [config.lora_dropout],
        'Freeze T5': [config.freeze_lm],
        'Min Training Loss': [statistics[0]],
        'Min Validation Loss': [statistics[1]],
        'Min Testing Loss': [statistics[2]],
    }
    pd.DataFrame(trial_dict).to_csv(
        os.path.join('/content/drive/MyDrive/DriveLM_project/DriveLM',
                     'multi_frame_results', timestr, 'multi_frame_results.csv'),
        index=False, header=True
    )

# ========================
# MAIN EXECUTION
# ========================
if __name__ == '__main__':
    timestr = time.strftime("%Y%m%d-%H%M%S")
    losses, val_losses = [], []
    min_train_loss, min_val_loss, best_model, epochs_ran = None, None, None, 0

    model = DriveVLM(config).to(device)
    print_trainable_parameters(model)

    # Tokenizer
    if config.lm == 'T5':
        processor = T5Tokenizer.from_pretrained('google-t5/t5-base')
        processor.add_tokens('<')
    else:
        processor = GPT2TokenizerFast.from_pretrained('gpt2-medium')
        processor.pad_token = processor.eos_token

    # datasets
    train_dset = MultiFrameDataset(
        os.path.join('/content/drive/MyDrive/DriveLM_project/DriveLM', 'data', 'multi_frame', 'multi_frame_train.json'),
        tokenizer=processor,
        transform=transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.Normalize((127.5, 127.5, 127.5), (127.5, 127.5, 127.5))
        ])
    )
    val_dset = MultiFrameDataset(
        os.path.join('/content/drive/MyDrive/DriveLM_project/DriveLM', 'data', 'multi_frame', 'multi_frame_val.json'),
        tokenizer=processor,
        transform=transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.Normalize((127.5, 127.5, 127.5), (127.5, 127.5, 127.5))
        ])
    )
    test_dset = MultiFrameDataset(
        os.path.join('/content/drive/MyDrive/DriveLM_project/DriveLM', 'data', 'multi_frame', 'multi_frame_test.json'),
        tokenizer=processor,
        transform=transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.Normalize((127.5, 127.5, 127.5), (127.5, 127.5, 127.5))
        ])
    )

    # dataloaders
    train_dataloader = DataLoader(train_dset, shuffle=True, batch_size=config.batch_size,
                                  num_workers=config.num_workers, collate_fn=train_dset.collate_fn)
    val_dataloader = DataLoader(val_dset, shuffle=True, batch_size=config.batch_size,
                                num_workers=config.num_workers, collate_fn=train_dset.collate_fn)
    test_dataloader = DataLoader(test_dset, shuffle=True, batch_size=config.batch_size,
                                 num_workers=config.num_workers, collate_fn=train_dset.collate_fn)

    if config.custom_train:
        if config.load_checkpoint:
            print(f"Resuming from checkpoint: {config.file_checkpoint}")
            model = load_model(model, config.file_checkpoint)
            # load stats
            with open(os.path.join('/content/drive/MyDrive/DriveLM_project/DriveLM',
                                   'multi_frame_results', config.file_checkpoint, 'stats.json'), 'r') as f:
                stats = json.load(f)
            min_train_loss, min_val_loss, losses, val_losses, epochs_ran = (
                stats['min train loss'], stats['min val loss'],
                stats['losses'], stats['val losses'], stats['epochs']
            )
            timestr = config.file_checkpoint
        else:
            os.mkdir(os.path.join('/content/drive/MyDrive/DriveLM_project/DriveLM',
                                  'multi_frame_results', timestr))

        lr = stats['learning rate'] if config.load_checkpoint else config.learning_rate
        min_train_loss, min_val_loss = custom_train(min_train_loss, min_val_loss, best_model, epochs_ran, lr)
        best_model = load_model(DriveVLM(config).to(device), 'latest_model')
        test_loss = val_model(test_dataloader, best_model)
        save_experiment([min_train_loss, min_val_loss, test_loss])
    else:
        train()


In [ ]:
while True:
    from IPython.display import HTML
    import time
    display(HTML("<script>window.scrollTo(0, document.body.scrollHeight);</script>"))
    time.sleep(60)


In [ ]:

!pip install peft
!pip install pycocoevalcap
!pip install pycocotools

In [ ]:
 def collate_fn_test(self, batch):

        q_texts, imgs, a_texts, img_paths = zip(*batch)

        imgs = torch.stack(list(imgs), dim=0)
        img_paths = list(img_paths)
        encodings = self.tokenizer(q_texts, padding=True, return_tensors="pt").input_ids.to(device)
        labels = self.tokenizer(a_texts, padding=True, return_tensors='pt').input_ids.to(device)

        return q_texts, encodings, imgs, labels, img_paths

In [ ]:
VIT_HIDDEN_STATE = 768
VIT_SEQ_LENGTH = 49

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    print(
        f"Trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )



class DriveVLMT5(nn.Module):

    def __init__(self, config):

        super().__init__()

        # Make tokenizer and text model
        if config.lm == 'T5-Base':
            self.model = T5ForConditionalGeneration.from_pretrained('google-t5/t5-base')
        else:
            self.model = T5ForConditionalGeneration.from_pretrained('google-t5/t5-large')

            # For quantization
            loftq_config = LoftQConfig(loftq_bits=8)
            # Create LoRA model
            lora_config = LoraConfig(
                r=64,
                lora_alpha=32,
                loftq_config=loftq_config,
                lora_dropout=0.05,
                bias='none',
                target_modules=['q', 'v']
            )
            self.model = get_peft_model(self.model, lora_config)

        hidden_size = self.model.config.d_model

        print('Trainable Parameters for LM model:')
        print_trainable_parameters(self.model)

        # Create instance for multi-view processor
        self.mvp = self.MultiViewProcessor(config.gpa_hidden_size, hidden_size, config.lm, freeze=True)

    class MultiViewProcessor(nn.Module):

        def __init__(self, gpa_hidden_size, hidden_size, lm, freeze=False):

            super().__init__()

            # Use ViT for image embeddings
            self.img_model = vit_b_32(weights='DEFAULT')
            self.lm = lm

            # Modal embedding to distinguish between image and text
            self.modal_embeddings = nn.Embedding(2, hidden_size)
            self.modal_embeddings.weight.data.normal_(mean=0.0, std=0.02)

            # If we are freezing the CLIP embeddings
            if freeze:
                for param in self.img_model.parameters():
                    param.requires_grad = False

            # Set matrices based on MIVC paper
            self.w = nn.Linear(in_features=gpa_hidden_size, out_features=1)
            self.Z = nn.Sequential(
                nn.Linear(in_features=VIT_HIDDEN_STATE * VIT_SEQ_LENGTH, out_features=gpa_hidden_size, bias=False),
                nn.Tanh()
            )
            self.G = nn.Sequential(
                nn.Linear(in_features=VIT_HIDDEN_STATE * VIT_SEQ_LENGTH, out_features=gpa_hidden_size, bias=False),
                nn.Sigmoid()
            )

            if self.lm != 'T5-Base':
              self.img_projection_layer = nn.Linear(in_features=VIT_HIDDEN_STATE, out_features=hidden_size)


        def gpa(self, img_embeddings):

            """"
            Calculates the gated-pooling attention score for the image embeddings
            :param img_embeddings: (6x768) dimensional
            :return single embedding of size (768,)
            """

            # Get weights for gated pooling attention
            gpa_weights = torch.softmax(self.w(self.Z(img_embeddings) * self.G(img_embeddings)), dim=0  )

            # Take a linear combination of all the image embeddings
            fused_embeddings = torch.sum(gpa_weights * img_embeddings, dim=0)

            return fused_embeddings

        def get_img_embedding(self, imgs):

            N = imgs.shape[0]

            # Process into patches (N x 6 x 49 x H)
            merged_embedding = torch.stack([self.img_model._process_input(img) for img in imgs], dim=0)

            # Concatenate the batch class tokens -> (N, 6, 50, H)
            batch_class_tokens = self.img_model.class_token.expand(merged_embedding.shape[1], -1, -1).repeat(N, 1, 1, 1)
            merged_embedding = torch.cat([batch_class_tokens, merged_embedding], dim=2)

            # Add positional embeddings and remove class token -> (N, 6, 49, H)
            merged_embedding += self.img_model.encoder.pos_embedding.repeat(N, 1, 1, 1)
            merged_embedding = merged_embedding[:, :, 1:]

            # Get merged embedding and reshape to 2D embedding -> (N, 1, 49, H)
            merged_embedding = torch.stack([self.gpa(embedding.flatten(start_dim=1)).reshape(VIT_SEQ_LENGTH,
                                            VIT_HIDDEN_STATE) for embedding in merged_embedding], dim=0)

            # Project to VL dimension -> (1, 49, H) (H is 512 for t5-small, 768 for t5-base)
            if self.lm != 'T5-Base':
              merged_embedding = self.img_projection_layer(merged_embedding)

            # Add modal type embedding to merged embedding
            merged_embedding += self.modal_embeddings(
                torch.ones((1, merged_embedding.shape[1]), dtype=torch.int, device=device))

            return merged_embedding

        def forward(self, text_enc, imgs, text_model):

            # Get the image embeddings (N x 1 x 49 x H)
            imgs_embedding = self.get_img_embedding(imgs)

            # Get the text embeddings (N x S x H)
            text_embeddings = text_model.get_input_embeddings()(text_enc)

            # Add modal embeddings to text
            text_embeddings += self.modal_embeddings(torch.zeros((1, text_embeddings.shape[1]), dtype=torch.int,
                                                                 device=device))

            # Concatenate embeddings -> (1 x S x 512)
            merged_embedding = torch.cat([text_embeddings, imgs_embedding], dim=1)

            return merged_embedding

    def forward(self, text_enc, imgs, labels=None):

        # Get the merged embeddings
        merged_embedding = self.mvp(text_enc, imgs, self.model)

        # If training include the labels
        return self.model(inputs_embeds=merged_embedding, labels=labels)

    def generate(self, text_enc, imgs, lidar=None):

        merged_embedding = self.mvp(text_enc, imgs, self.model)

        attention_mask = torch.ones(merged_embedding.shape[:2], dtype=torch.long, device=device)
        decoder_input_ids = torch.ones((merged_embedding.shape[0], 1), dtype=torch.long, device=device)*self.model.config.decoder_start_token_id
        output_ids = self.model.generate(attention_mask=attention_mask, decoder_input_ids=decoder_input_ids, inputs_embeds=merged_embedding, max_length=512, early_stopping=True)

        return output_ids

In [ ]:
Config = namedtuple('Instance', ['batch_size', 'gpa_hidden_size', 'model_name', 'lm'])

config = Config(
    batch_size = 16,
    gpa_hidden_size = 128,
    model_name = 'T5-Large-Q',
    lm = 'T5-Large'
)

In [ ]:
import glob

# List all subfolders in your multi_frame_results
glob.glob("/content/drive/MyDrive/DriveLM_project/DriveLM/multi_frame_results/*")


['/content/drive/MyDrive/DriveLM_project/DriveLM/multi_frame_results/20250804-055948',
 '/content/drive/MyDrive/DriveLM_project/DriveLM/multi_frame_results/20250804-060259',
 '/content/drive/MyDrive/DriveLM_project/DriveLM/multi_frame_results/20250804-061435',
 '/content/drive/MyDrive/DriveLM_project/DriveLM/multi_frame_results/20250804-063559',
 '/content/drive/MyDrive/DriveLM_project/DriveLM/multi_frame_results/20250804-063955']

In [ ]:
import glob

glob.glob("/content/drive/MyDrive/DriveLM_project/DriveLM/multi_frame_results/20250804-055948/*")





[]

In [ ]:
!ls -l "/content/drive/MyDrive/DriveLM_project/DriveLM/multi_frame_results/20250804-063955"


total 0


In [ ]:
import os
import json
import torch
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm as progress_bar
from torchvision import transforms
from torch.utils.data import DataLoader
from transformers import T5Tokenizer

# ===== Your trained run folder =====
checkpoint_run = "/content/drive/MyDrive/DriveLM_project/DriveLM/multi_frame_results/20250804-063955"

# ===== Paths =====
base_dir = "/content/drive/MyDrive/DriveLM_project/DriveLM"
ckpt_dir = os.path.join(base_dir, "multi_frame_results", checkpoint_run)

# ===== Load processors and model =====
model = DriveVLMT5(config).to(device)

if config.lm == 'T5-Base':
    processor = T5Tokenizer.from_pretrained('google-t5/t5-base')
else:
    processor = T5Tokenizer.from_pretrained('google-t5/t5-large')

processor.add_tokens('<')

# Load trained checkpoint
model.load_state_dict(torch.load(os.path.join(ckpt_dir, 'latest_model.pth'), map_location=device))

# ===== Dataset & DataLoader =====
test_dset = MultiFrameDataset(
    input_file=os.path.join(base_dir, "data", "multi_frame", "prompts_for_drivelm.json"),
    tokenizer=processor,
    transform=transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Normalize((127.5, 127.5, 127.5), (127.5, 127.5, 127.5))
    ])
)
test_dloader = DataLoader(test_dset, shuffle=True, batch_size=1, drop_last=True, collate_fn=test_dset.collate_fn_test)

# ===== Image ID mapping =====
with open(os.path.join(base_dir, "data", "multi_frame", "image_id.json")) as f:
    image_id_dict = json.load(f)

# ===== Inference function =====
def val_model(dloader):
    model.eval()
    test_data = []

    with torch.no_grad():
        for idx, (q_texts, encodings, imgs, labels, img_paths) in progress_bar(enumerate(dloader), total=len(dloader)):
            outputs = model.generate(encodings, imgs)
            text_outputs = [processor.decode(output, skip_special_tokens=True) for output in outputs]

            for image_path, q_text, text_output in zip(img_paths, q_texts, text_outputs):
                img_key = image_path[0]
                test_data.append({'ImgID and Question': f"{img_key} , {q_text}", 'caption': text_output})

                fig, axes = plt.subplots(3, 2, figsize=(12, 8))
                for ax, img_p in zip(axes.flat, img_paths[0]):
                    img = mpimg.imread(img_p)
                    ax.imshow(img)
                    ax.set_title(os.path.basename(img_p))
                    ax.axis('off')

                fig.suptitle(f"Q: {q_text}\nA: {text_output}", fontsize=14)
                plt.tight_layout()
                plt.show()

    # Save predictions
    out_path = os.path.join(ckpt_dir, 'predictions-test.json')
    with open(out_path, 'w') as f:
        json.dump(test_data, f, indent=2)
    print(f"✅ Predictions saved to {out_path}")

# ===== Run inference =====
val_model(test_dloader)


Trainable Parameters for LM model:
Trainable params: 18874368 || all params: 756542464 || trainable%: 2.494819378704432


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/DriveLM_project/DriveLM/multi_frame_results/20250804-063955/latest_model.pth'